In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import operator 
from pmdarima.arima import auto_arima
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
from datetime import date
import operator
from matplotlib import pyplot

In [2]:
confirmed_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
coloco = ['Lat','Long']
confirmed_cases_global.drop(coloco, inplace=True, axis=1)
confirmed_cases_global.head()


,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,607,665,714,784,840,906,933,996,1026,1092
1,NaN,Albania,0,0,0,0,0,0,0,0,...,446,467,475,494,518,539,548,562,584,609
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,1914,1983,2070,2160,2268,2418,2534,2629,2718,2811
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,638,646,659,673,673,696,704,713,717,717
4,NaN,Angola,0,0,0,0,0,0,0,0,...,19,19,19,19,19,19,24,24,24,24


In [3]:
confirmed_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
confirmed_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region', 'Long_':'Long' }, inplace=True)
coloc = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key','Lat','Long']
confirmed_cases_US.drop(coloc, inplace=True, axis=1)
confirmed_cases_US.head()


,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,American Samoa,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,0,0,0,0,0,0,0,0,...,133,133,133,135,135,136,136,136,136,136
2,Northern Mariana Islands,US,0,0,0,0,0,0,0,0,...,11,11,11,13,13,13,14,14,14,14
3,Puerto Rico,US,0,0,0,0,0,0,0,0,...,897,903,923,974,1043,1068,1118,1213,1252,1298
4,Virgin Islands,US,0,0,0,0,0,0,0,0,...,51,51,51,51,51,51,53,53,53,53


In [4]:
confirmed_cases_US.loc[confirmed_cases_US['Province/State'] == 'Rhode Island']

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
2316,Rhode Island,US,0,0,0,0,0,0,0,0,...,39,39,39,49,50,57,57,59,67,68
2317,Rhode Island,US,0,0,0,0,0,0,0,0,...,169,169,169,244,249,299,299,316,366,386
2318,Rhode Island,US,0,0,0,0,0,0,0,0,...,74,74,74,93,94,103,103,103,107,110
2319,Rhode Island,US,0,0,0,0,0,0,0,0,...,1278,1278,1278,1957,2072,2703,2703,3014,3491,3727
2320,Rhode Island,US,0,0,0,0,0,0,0,0,...,140,140,140,152,162,225,183,184,198,209
3188,Rhode Island,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3240,Rhode Island,US,0,0,0,0,0,0,0,0,...,965,965,1551,756,902,790,1146,1030,861,1000


In [5]:
confirmed_cases_US=confirmed_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()
#.apply(lambda x : x.sum().reset_index())

In [6]:
confirmed_cases_US.loc[confirmed_cases_US['Province/State'] == 'Rhode Island']

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
46,Rhode Island,US,0,0,0,0,0,0,0,0,...,2665,2665,3251,3251,3529,4177,4491,4706,5090,5500


In [7]:
deaths_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
colo = ['Lat','Long']
deaths_cases_global.drop(colo, inplace=True, axis=1)
deaths_cases_global.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,18,21,23,25,30,30,30,33,36,36
1,NaN,Albania,0,0,0,0,0,0,0,0,...,23,23,24,25,26,26,26,26,26,26
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,293,313,326,336,348,364,367,375,384,392
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,29,29,31,33,33,35,35,36,37,37
4,NaN,Angola,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [8]:
deaths_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
deaths_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region','Long_':'Long' }, inplace=True)
col = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key', 'Population','Long','Lat']
deaths_cases_US.drop(col, inplace=True, axis=1)
deaths_cases_US.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,American Samoa,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
2,Northern Mariana Islands,US,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
3,Puerto Rico,US,0,0,0,0,0,0,0,0,...,44,45,45,51,56,58,60,62,63,64
4,Virgin Islands,US,0,0,0,0,0,0,0,0,...,1,1,1,1,1,2,3,3,3,3


In [9]:
deaths_cases_US=deaths_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()

In [10]:
confirmed_cases= pd.concat([confirmed_cases_global, confirmed_cases_US], axis=0, sort=False)
confirmed_cases

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,607,665,714,784,840,906,933,996,1026,1092
1,NaN,Albania,0,0,0,0,0,0,0,0,...,446,467,475,494,518,539,548,562,584,609
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,1914,1983,2070,2160,2268,2418,2534,2629,2718,2811
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,638,646,659,673,673,696,704,713,717,717
4,NaN,Angola,0,0,0,0,0,0,0,0,...,19,19,19,19,19,19,24,24,24,24
5,NaN,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,21,23,23,23,23,23,23,23,23,23
6,NaN,Argentina,0,0,0,0,0,0,0,0,...,2142,2208,2277,2443,2571,2669,2758,2839,2941,3031
7,NaN,Armenia,0,0,0,0,0,0,0,0,...,1013,1039,1067,1111,1159,1201,1248,1291,1339,1401
8,Australian Capital Territory,Australia,0,0,0,0,0,0,0,0,...,103,102,103,103,103,103,103,103,103,103
9,New South Wales,Australia,0,0,0,0,3,4,4,4,...,2857,2863,2870,2886,2897,2926,2926,2926,2926,2926


In [11]:
deaths_cases= pd.concat([deaths_cases_global, deaths_cases_US], axis=0, sort=False)
deaths_cases

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,18,21,23,25,30,30,30,33,36,36
1,NaN,Albania,0,0,0,0,0,0,0,0,...,23,23,24,25,26,26,26,26,26,26
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,293,313,326,336,348,364,367,375,384,392
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,29,29,31,33,33,35,35,36,37,37
4,NaN,Angola,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
5,NaN,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,2,2,2,2,3,3,3,3,3,3
6,NaN,Argentina,0,0,0,0,0,0,0,0,...,90,97,102,111,115,123,129,132,136,147
7,NaN,Armenia,0,0,0,0,0,0,0,0,...,13,14,16,17,18,19,20,20,22,24
8,Australian Capital Territory,Australia,0,0,0,0,0,0,0,0,...,2,2,2,3,3,3,3,3,3,3
9,New South Wales,Australia,0,0,0,0,0,0,0,0,...,24,25,25,25,25,26,26,26,26,26


In [12]:
recoveries_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
coloco = ['Lat','Long']
recoveries_cases.drop(coloco, inplace=True, axis=1)
recoveries_cases.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,32,32,40,43,54,99,112,131,135,150
1,NaN,Albania,0,0,0,0,0,0,0,0,...,217,232,248,251,277,283,302,314,327,345
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,591,601,691,708,783,846,894,1047,1099,1152
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,128,128,128,169,169,191,205,235,248,282
4,NaN,Angola,0,0,0,0,0,0,0,0,...,4,4,5,5,5,5,6,6,6,6


In [13]:
colnames = confirmed_cases.columns.values
colnames

array(['Province/State', 'Country/Region', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20',
       '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20',
       '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20',
       '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20',
       '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20',
       '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20',
       '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20',
       '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20',
       '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20',
       '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20',
       '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20', '4/10/20',
       '4/11/

In [14]:
colonames = recoveries_cases.columns.values
colonames

array(['Province/State', 'Country/Region', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/20',
       '1/30/20', '1/31/20', '2/1/20', '2/2/20', '2/3/20', '2/4/20',
       '2/5/20', '2/6/20', '2/7/20', '2/8/20', '2/9/20', '2/10/20',
       '2/11/20', '2/12/20', '2/13/20', '2/14/20', '2/15/20', '2/16/20',
       '2/17/20', '2/18/20', '2/19/20', '2/20/20', '2/21/20', '2/22/20',
       '2/23/20', '2/24/20', '2/25/20', '2/26/20', '2/27/20', '2/28/20',
       '2/29/20', '3/1/20', '3/2/20', '3/3/20', '3/4/20', '3/5/20',
       '3/6/20', '3/7/20', '3/8/20', '3/9/20', '3/10/20', '3/11/20',
       '3/12/20', '3/13/20', '3/14/20', '3/15/20', '3/16/20', '3/17/20',
       '3/18/20', '3/19/20', '3/20/20', '3/21/20', '3/22/20', '3/23/20',
       '3/24/20', '3/25/20', '3/26/20', '3/27/20', '3/28/20', '3/29/20',
       '3/30/20', '3/31/20', '4/1/20', '4/2/20', '4/3/20', '4/4/20',
       '4/5/20', '4/6/20', '4/7/20', '4/8/20', '4/9/20', '4/10/20',
       '4/11/

In [15]:
confirmed = confirmed_cases.loc[:, colnames[4]:colnames[-1]]
deaths = deaths_cases.loc[:, colnames[4]:colnames[-1]]
recoveries = recoveries_cases.loc[:, colonames[4]:colonames[-1]]

In [16]:
recoveries

,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,0,0,0,0,0,0,0,0,0,0,...,32,32,40,43,54,99,112,131,135,150
1,0,0,0,0,0,0,0,0,0,0,...,217,232,248,251,277,283,302,314,327,345
2,0,0,0,0,0,0,0,0,0,0,...,591,601,691,708,783,846,894,1047,1099,1152
3,0,0,0,0,0,0,0,0,0,0,...,128,128,128,169,169,191,205,235,248,282
4,0,0,0,0,0,0,0,0,0,0,...,4,4,5,5,5,5,6,6,6,6
5,0,0,0,0,0,0,0,0,0,0,...,0,0,3,3,3,3,3,3,3,7
6,0,0,0,0,0,0,0,0,0,0,...,468,515,559,596,631,666,685,709,737,840
7,0,0,0,0,0,0,0,0,0,0,...,197,211,265,297,358,402,523,545,580,609
8,0,0,0,0,0,0,0,0,0,0,...,59,59,72,72,82,81,88,88,88,88
9,0,0,0,0,0,0,2,2,2,2,...,4,4,4,4,4,1379,1379,1379,1379,1379


In [17]:
time_date = confirmed.columns.values

In [18]:
all_days = np.array([i for i in range(len(time_date))]).reshape(-1, 1)

# Predictions for days ahead

In [19]:
pred_days_ahead = 2
#pred_days_ahead = 7
#pred_days_ahead = 30
predictions_ahead = np.array([i for i in range(len(time_date)+pred_days_ahead)]).reshape(-1, 1)

predictions_ahead_plus_day = np.array([i for i in range(len(time_date)+pred_days_ahead+1)]).reshape(-1, 1)

In [20]:
start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
predictions_ahead_dates = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day))]

In [21]:
confirmed_cases = confirmed_cases[~confirmed_cases['Country/Region'].isin(['Canada','Central African Republic','Chad'])]
confirmed_cases
#ignore Canada for now

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,607,665,714,784,840,906,933,996,1026,1092
1,NaN,Albania,0,0,0,0,0,0,0,0,...,446,467,475,494,518,539,548,562,584,609
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,1914,1983,2070,2160,2268,2418,2534,2629,2718,2811
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,638,646,659,673,673,696,704,713,717,717
4,NaN,Angola,0,0,0,0,0,0,0,0,...,19,19,19,19,19,19,24,24,24,24
5,NaN,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,21,23,23,23,23,23,23,23,23,23
6,NaN,Argentina,0,0,0,0,0,0,0,0,...,2142,2208,2277,2443,2571,2669,2758,2839,2941,3031
7,NaN,Armenia,0,0,0,0,0,0,0,0,...,1013,1039,1067,1111,1159,1201,1248,1291,1339,1401
8,Australian Capital Territory,Australia,0,0,0,0,0,0,0,0,...,103,102,103,103,103,103,103,103,103,103
9,New South Wales,Australia,0,0,0,0,3,4,4,4,...,2857,2863,2870,2886,2897,2926,2926,2926,2926,2926


In [22]:
#countries2 = ['Greece', 'Italy', 'Spain', 'Turkey','Cyprus', 'Switzerland']
#countries= confirmed_cases.iloc[35:46,1].tolist()#keep all  country names in a list
#countries=countries+countries2
#province=confirmed_cases.iloc[35:46,0].tolist()#province list with nan
#prov=[float("NaN")]*6
#province=province+prov
#countries1=confirmed_cases['Country/Region'].tolist()
#countries2= [x for x in countries1 if (x != 'Canada' and x != 'Uganda')]
#province=confirmed_cases['Province/State'].tolist()
#countries = ['Algeria','France', 'France','Greece']
#province=[float("NaN"),'Martinique', float("NaN"),float("NaN")]
#province
#'French Guiana','French Polynesia','Guadeloupe','Mayotte','New Caledonia','Reunion','Saint Barthelemy',
       #'St Martin','
#countries=['Andorra', 'Algeria', 'Australia']

#province=[float("NaN"),float("NaN"), 'New South Wales']
#province=[confirmed_cases['Province/State'][x] for x in range(len(confirmed_cases['Province/State'])) if (confirmed_cases['Country/Region'][x] != 'Canada' and confirmed_cases['Country/Region'][x] != 'Uganda') ]

In [23]:
countries=confirmed_cases['Country/Region'].tolist()
#countries.append("US")
#countries= [x for x in countries1 if (x != 'Canada')]
#print(countries)
#countries=['Uganda']
#province=[float('NaN')]
province=confirmed_cases['Province/State'].tolist()
#province.append(float('NaN'))
#print(province)
#province=[float("NaN"),float("NaN")]
#province=[confirmed_cases['Province/State'][x] for x in range(len(confirmed_cases['Province/State'])) if (confirmed_cases['Country/Region'][x] != 'Canada') ]


In [24]:
prediction_models = dict()
prediction_models_confirmed = {'SVR': 0}
prediction_models_deaths = {'SVR': 0}
prediction_models_recoveries = {'SVR': 0}



In [25]:
# Prediction Models

from sklearn import model_selection
from sklearn.svm import SVR
def SVM_regression(train_x, train_y, test_x, test_y, X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
   # c = [0.01, 0.1, 1]
   # gamma = [0.01, 0.1, 1]
   #epsilon = [0.01, 0.1, 1]
   #shrinking = [True, False]
    #degree = [3, 4, 5, 6]
  #grid_parameters = {'C': c, 'gamma' : gamma ,'epsilon': epsilon, 'shrinking' : shrinking, 'degree':degree} 
   #regressor = RandomizedSearchCV(SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1, C=0.1), grid_parameters, scoring='neg_mean_squared_error', 
    #                               cv=5)
    
    if (subpopulation == 'confirmed'):
         degree = 3
    elif (subpopulation == 'deaths'):
        degree = 4
    else :
        degree = 5


    regressor = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=degree, C=0.1)
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['SVR'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['SVR'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['SVR'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions) 
    

In [26]:
from sklearn.ensemble import RandomForestRegressor
def RF_regression(train_x, train_y, test_x, test_y,X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    grid_parameters = {'n_estimators': n_estimators}
#    regressor = RandomizedSearchCV(RandomForestRegressor(), grid_parameters, scoring='neg_mean_squared_error', 
#                                   cv=3)
    
    regressor = RandomForestRegressor(n_estimators = 200, random_state = 42) 
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)


In [27]:
# Gradient Boosted Regression Trees (GBRT) 
from sklearn.ensemble import GradientBoostingRegressor

def Grad_Boost_regression(train_x, train_y, test_x, test_y,X_test,subpopulation, X_train, Y_train):
   
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    loss = ['ls','lad','huber']
    grid_parameters = {'n_estimators': n_estimators}
  #  regressor = RandomizedSearchCV(GradientBoostingRegressor(learning_rate=0.1), grid_parameters, scoring='neg_mean_squared_error', 
 #                                  cv=3)
    
    regressor =  GradientBoostingRegressor(n_estimators=500, learning_rate=0.1,
                                            max_depth=1, random_state=0, loss='huber')
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)



In [28]:
#check if a timeseries is stationary
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()     
    rolstd = timeseries.rolling(12).std()

    #Plot rolling statistics:
    #orig = plt.plot(timeseries, color='blue',label='Original')
    #mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    #std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    #plt.legend(loc='best')
    #plt.title('Rolling Mean & Standard Deviation')
    #plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    #print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput
    

In [29]:
def pred_with_arima(timeseries,n_days_ahead):
    
    #find the optimal d 
    diff=timeseries-timeseries.shift(1)
    for d in range(1,10):
        if (test_stationarity(diff.dropna())['p-value']<0.1):
            #print (d) 
            break
        else: 
            d=d+1 
            diff=diff-diff.shift(1)
    #use the arima model to make predictions, use original timeseries with optimal d:
    
    model = pm.auto_arima(timeseries, start_p=0, start_q=0,
                             max_p=4, max_q=4,
                              seasonal=False,
                             d=d, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=False)
    print(model)
    forecasts, conf_int =model.predict(n_days_ahead,return_conf_int=True ,alpha=0.05)
    return forecasts , conf_int # returns a tuple, array of forecasts and all conf int

In [30]:
def Arima_Analysis(region,pred_days_ahead):
   #create a dataframe for timeseries analysis
    date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
    df = pd.DataFrame(date_rng, columns=['date'])
    df['cases']=np.transpose(region) #this is a dataframe with dates and cases.
    #df['datetime'] = pd.to_datetime(df['date'])
    df = df.set_index('date')
    #df.drop(['date'], axis=1, inplace=True)
    #df.plot( style='k.')
    #pyplot.show()
    df=df[df['cases'] != 0] #remove zeroes, they do not help for analysis. 
    if (len(df)>1):
        if(df['cases'][-1]-df['cases'][0]>5):
            pred_ts,conf_int=pred_with_arima(df['cases'],pred_days_ahead)  #.tolist() #this is an array converted to a list
        else:
            pred_ts=[df['cases'][-1]+1]*pred_days_ahead
            conf_int=[np.array([df['cases'][-1],df['cases'][-1]+2])]*pred_days_ahead
    else:
        pred_ts=np.zeros(pred_days_ahead)
        conf_int=[np.zeros(pred_days_ahead)] * pred_days_ahead
    return pred_ts, conf_int

In [31]:
# region = confirmed_cases.loc[(confirmed_cases['Country/Region'] == 'Canada') & (confirmed_cases['Province/State'] == 'Alberta')] 
# region =  np.array(region.loc[:, colnames[4]:colnames[-1]])   
# date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
# df = pd.DataFrame(date_rng, columns=['date'])
# df['cases']=np.transpose(region)
# df = df.set_index('date')
# df=df[df['cases'] != 0]

In [32]:
#prediction, confidence= Arima_Analysis(region,2)

In [33]:
#pred_with_arima(df['cases'],2)[0]
#prediction

In [34]:
import operator
def subpopulation_Analysis(cases,country,province, all_days,predictions_ahead,subpopulation, pred_days_ahead):
    
    pred  = list()
    if pd.isnull(province):
        region = cases.loc[(cases['Country/Region'] == country) & (pd.isnull(cases['Province/State']))]
    else :
        region = cases.loc[(cases['Country/Region'] == country) & (cases['Province/State'] == province)]
    print(region)
    
    reg =  np.array(region.loc[:, colnames[4]:colnames[-1]])
    region =  np.array(region.loc[:, colnames[4]:colnames[-1]]).reshape(-1, 1)
    
    #print(reg.item())
    print(reg[-1][-1])
    
    #train_x, test_x, train_y, test_y = model_selection.train_test_split(all_days,region,test_size = 0.04,random_state = 42,shuffle=False)
    
    #pred_svm =  SVM_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_svm)
    
    #TODO: a for loop
    daily_increase_threshold1=np.absolute((reg[-1][-1]-reg[-1][-2])/reg[-1][-2])*100
    daily_increase_threshold2=np.absolute((reg[-1][-2]-reg[-1][-3])/reg[-1][-3])*100
    daily_increase_threshold3=np.absolute((reg[-1][-3]-reg[-1][-4])/reg[-1][-4])*100
    daily_increase_threshold4=np.absolute((reg[-1][-4]-reg[-1][-5])/reg[-1][-5])*100
    daily_increase_bound=1000
    daily_increase_threshold = max(daily_increase_threshold1,daily_increase_threshold2,daily_increase_threshold3,daily_increase_threshold4)
    if (daily_increase_threshold>daily_increase_bound):
        best_pred_ahead = 0  
        region = region
        best_pred = 0
        best_conf_int = np.zeros(2)
        reject_country = 1
        return(best_pred_ahead, region, best_pred, best_conf_int, reject_country)
     
    
    pred_arima, conf_int=Arima_Analysis(reg,pred_days_ahead)
    #pred.append(pred_svm)
    #pred_RF = RF_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_RF)

    
   # pred_gb =  Grad_Boost_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
   # pred.append(pred_gb)


    if (subpopulation == 'confirmed'): 
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_confirmed.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    elif (subpopulation == 'deaths'):
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_deaths.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    else :
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_recoveries.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
        
    

    best_pred_ahead = best_pred[-1]
    best_conf_int=conf_int[-1]
    #rint(best_conf_int)
    #rint("GNTM")
    best_conf_int[0]=max(reg[-1][-1], best_conf_int[0])
    if (best_conf_int[0]>best_conf_int[1]):
        best_conf_int[1]=2*reg[-1][-1]
  # print(best_conf_int)
    print(best_pred_ahead)
    print("GNTM")
    if(best_pred_ahead<reg[-1][-1]):
        best_pred_ahead= reg[-1][-1]+pred_days_ahead
    
    reject_country = 0
    print(best_pred_ahead)
    return best_pred_ahead, region, best_pred, best_conf_int, reject_country

In [35]:
pred_file = str(2)+"day_prediction_" +  date.today().strftime('%Y-%m-%d') + ".csv"
#pred_file = str(7)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + ".csv"
#pred_file = str(30)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + ".csv"
pred_file

'2day_prediction_2020-04-22.csv'

In [36]:
columns = ['Province/State','Country','Target/Date','N','low95N','high95N',
          'R','low95R','high95R','D','low95D','high95D','T','low95T','high95T',
           'M','low95M','high95M','C','low95C','high95C']

            

k = 0
for co in range(len(countries)):
    
    ignore_recoveries = 0    
    
    total_predictions = pd.DataFrame(columns = columns)
    
    k = k + 1
       
    best_pred_confirmed, region_con, best_pred_con, conf_int_con, reject_country = subpopulation_Analysis(confirmed_cases,countries[co],province[co],
                                                 all_days,predictions_ahead,"confirmed",pred_days_ahead)
    
    if (reject_country==1):
        continue
        
    best_pred_deaths, region_d, best_pred_d, conf_int_d, reject_country = subpopulation_Analysis(deaths_cases,countries[co],province[co],
                                              all_days,predictions_ahead,"deaths",pred_days_ahead)
    if (reject_country==1):
        continue
        
    if ((countries[co] == 'US') & (pd.notnull(province[co]))):
        ignore_recoveries=1   
    else:
        best_pred_recoveries, region_r, best_pred_r, conf_int_r, reject_country = subpopulation_Analysis(recoveries_cases,countries[co],province[co],
                                                  all_days,predictions_ahead,"recoveries",pred_days_ahead)
    
    if (reject_country==1):
        continue
    
    # Mortality rate
    if (conf_int_con[0]==0):
        mort_pred=0
        mort_pred_low=0
        mort_pred_up=0
    else:
        mort_pred  = best_pred_deaths/best_pred_confirmed
        mort_pred_low= conf_int_d[0]/conf_int_con[1]
        mort_pred_up=conf_int_d[1]/conf_int_con[0]
    
    #fix province
    province_name=''
    if (pd.notnull(province[co])) :
        province_name=province[co]
    
    if (ignore_recoveries==0):
        R_value=int(best_pred_recoveries) 
        low_R=int(conf_int_r[0])
        high_R=int(conf_int_r[1])
    else:
        R_value=''
        low_R=''
        high_R=''
    #Critical Cases:
    #Assume that 5% of all cases are critical and need hospitalization:
    serious_percentage=0.05
    critical=serious_percentage*best_pred_confirmed
    low_critical=serious_percentage*conf_int_con[0]
    high_critical=serious_percentage*conf_int_con[1]
    
    total_predictions = total_predictions.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed) ,
                                                'low95N':int(conf_int_con[0]),
                                                'high95N': int(conf_int_con[1]),
                                                'R': R_value ,
                                                'low95R' : low_R,
                                                'high95R' : high_R,
                                                'D': int(best_pred_deaths) ,
                                                'low95D' : int(conf_int_d[0]),
                                                'high95D': int(conf_int_d[1]),
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred,
                                                'low95M': mort_pred_low,
                                                'high95M': mort_pred_up ,
                                                'C': int(critical) ,
                                                'low95C':  int(low_critical) ,
                                                'high95C': int(high_critical) },ignore_index = True)
    
    if (k==1):
        header = True
    else:
        header = False
    
    total_predictions.to_csv(pred_file, sep=',',index = False,  mode='a', header = header)
    

  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
0            NaN    Afghanistan        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  4/12/20  4/13/20  4/14/20  4/15/20  \
0        0        0        0  ...      607      665      714      784   

   4/16/20  4/17/20  4/18/20  4/19/20  4/20/20  4/21/20  
0      840      906      933      996     1026     1092  

[1 rows x 93 columns]
1092
Fit ARIMA: (0, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=499.704, BIC=503.755, Time=0.077 seconds
Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=458.564, BIC=464.640, Time=0.145 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=455.867, BIC=463.968, Time=0.176 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=451.205, BIC=461.332, Time=0.261 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=453.163, BIC=465.316, Time=0.430 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=462.057, BIC=

Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=287.931, BIC=293.215, Time=0.087 seconds
Fit ARIMA: (0, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=287.606, BIC=294.651, Time=0.120 seconds
Fit ARIMA: (0, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=288.848, BIC=297.654, Time=0.266 seconds
Fit ARIMA: (0, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=290.402, BIC=300.969, Time=0.252 seconds
Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=284.014, BIC=289.298, Time=0.062 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=285.978, BIC=293.023, Time=0.157 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=287.705, BIC=296.511, Time=0.246 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=289.687, BIC=300.254, Time=0.355 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=291.435, BIC=303.764, Time=0.631 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=285.989, BIC=293.033, Time=0.096 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (const

Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=510.711, BIC=518.741, Time=0.322 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=513.439, BIC=523.475, Time=0.343 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=515.357, BIC=527.401, Time=0.498 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=514.180, BIC=528.231, Time=0.401 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=514.076, BIC=522.106, Time=0.111 seconds
Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=514.258, BIC=524.294, Time=0.261 seconds
Fit ARIMA: (2, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=512.947, BIC=524.991, Time=0.647 seconds
Fit ARIMA: (2, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=513.460, BIC=527.512, Time=0.758 seconds
Fit ARIMA: (3, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=515.426, BIC=525.463, Time=0.187 seconds
Fit ARIMA: (3, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=515.645, BIC=527.689, Time=0.634 seconds
Fit ARIMA: (3, 2, 2)x(0, 0, 0, 0) (const

Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=377.788, BIC=387.247, Time=0.138 seconds
Fit ARIMA: (2, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=378.687, BIC=390.038, Time=0.262 seconds
Fit ARIMA: (2, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=380.451, BIC=393.693, Time=0.332 seconds
Fit ARIMA: (3, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=377.883, BIC=387.342, Time=0.142 seconds
Fit ARIMA: (3, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=378.108, BIC=389.459, Time=0.504 seconds
Fit ARIMA: (3, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=380.104, BIC=393.347, Time=0.553 seconds
Fit ARIMA: (4, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=379.663, BIC=391.014, Time=0.178 seconds
Fit ARIMA: (4, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=381.569, BIC=394.812, Time=0.355 seconds
Total fit time: 4.037 seconds
ARIMA(maxiter=50, method='lbfgs', order=(2, 2, 0), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,

Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=111.812, BIC=122.072, Time=0.595 seconds
Total fit time: 5.481 seconds
ARIMA(maxiter=50, method='lbfgs', order=(0, 1, 0), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,
      with_intercept=True)
25.4375
GNTM
25.4375
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
4            NaN         Angola        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  4/12/20  4/13/20  4/14/20  4/15/20  \
4        0        0        0  ...        2        2        2        2   

   4/16/20  4/17/20  4/18/20  4/19/20  4/20/20  4/21/20  
4        2        2        2        2        2        2  

[1 rows x 93 columns]
2
3
GNTM
3
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
4            NaN         Angola        0        0        0        0        0   

   1

...

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in longlong_scalars
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in longlong_scalars



Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=167.850, BIC=172.516, Time=0.212 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=169.356, BIC=175.578, Time=0.478 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=169.578, BIC=177.355, Time=0.216 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=169.753, BIC=179.085, Time=0.584 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=182.887, BIC=187.553, Time=0.056 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=169.499, BIC=175.720, Time=0.426 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=171.750, BIC=179.527, Time=0.385 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=173.109, BIC=182.441, Time=0.625 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=171.816, BIC=182.704, Time=0.935 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=179.426, BIC=185.647, Time=0.086 seconds
Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (cons

Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=1092.009, BIC=1099.407, Time=0.123 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=1087.091, BIC=1096.954, Time=0.474 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=1083.838, BIC=1096.167, Time=0.438 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=1083.386, BIC=1098.182, Time=0.610 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=1088.956, BIC=1096.354, Time=0.121 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=1084.217, BIC=1094.081, Time=0.264 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=1086.214, BIC=1098.543, Time=0.290 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=1084.548, BIC=1099.343, Time=0.876 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=1083.180, BIC=1100.441, Time=1.152 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=1084.015, BIC=1093.879, Time=0.153 seconds
Fit ARIMA: (2, 2, 1)

Fit ARIMA: (1, 3, 1)x(0, 0, 0, 0) (constant=True); AIC=88.278, BIC=92.261, Time=0.325 seconds
Fit ARIMA: (1, 3, 2)x(0, 0, 0, 0) (constant=True); AIC=78.618, BIC=83.597, Time=0.601 seconds
Fit ARIMA: (1, 3, 3)x(0, 0, 0, 0) (constant=True); AIC=80.016, BIC=85.991, Time=0.533 seconds
Fit ARIMA: (1, 3, 4)x(0, 0, 0, 0) (constant=True); AIC=73.181, BIC=80.151, Time=0.813 seconds
Fit ARIMA: (2, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=82.602, BIC=86.585, Time=0.093 seconds
Fit ARIMA: (2, 3, 1)x(0, 0, 0, 0) (constant=True); AIC=69.365, BIC=74.344, Time=0.347 seconds
Fit ARIMA: (2, 3, 2)x(0, 0, 0, 0) (constant=True); AIC=65.600, BIC=71.574, Time=0.411 seconds
Fit ARIMA: (2, 3, 3)x(0, 0, 0, 0) (constant=True); AIC=67.505, BIC=74.475, Time=0.542 seconds
Fit ARIMA: (3, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=75.610, BIC=80.589, Time=0.183 seconds
Fit ARIMA: (3, 3, 1)x(0, 0, 0, 0) (constant=True); AIC=66.865, BIC=72.839, Time=0.590 seconds
Fit ARIMA: (3, 3, 2)x(0, 0, 0, 0) (constant=True); AIC=67.63

Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=351.898, BIC=360.216, Time=0.321 seconds
Fit ARIMA: (2, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=353.792, BIC=363.773, Time=0.563 seconds
Fit ARIMA: (2, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=353.824, BIC=365.469, Time=0.840 seconds
Fit ARIMA: (3, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=354.186, BIC=362.504, Time=0.159 seconds
Fit ARIMA: (3, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=353.664, BIC=363.646, Time=0.641 seconds
Fit ARIMA: (3, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=357.111, BIC=368.756, Time=0.776 seconds
Fit ARIMA: (4, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=355.648, BIC=365.629, Time=0.180 seconds
Fit ARIMA: (4, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=357.521, BIC=369.166, Time=0.312 seconds
Total fit time: 8.056 seconds
ARIMA(maxiter=50, method='lbfgs', order=(0, 2, 1), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(0, 0, 0, 0),
      start_params=None, suppress_warnings=True, trend=None,